<a href="https://colab.research.google.com/github/deRockerTom/dataFrenchWords/blob/main/compressFrenchFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/deRockerTom/dataFrenchWords

Cloning into 'dataFrenchWords'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 19 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [3]:
def convertToBinary(number):
  binary = ''
  while number > 0:
    binary = str(number % 2) + binary
    number = number // 2
  return binary
def convertToTenBits(bits):
  n = len(bits)
  if n < 10:
    k = 10 - n
    return str('0') * k + bits
  elif n == 10:
    return bits
  else:
    print("Attention : un des encodages est trop grand (>10)")

convertToTenBits(convertToBinary(255))

'0011111111'

In [19]:
import csv

# open csv
dic = {}
with open('dataFrenchWords/french-word-list-verbs.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

with open('dataFrenchWords/french-word-list-adjectives.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

with open('dataFrenchWords/french-word-list-nouns.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

dic = dict(sorted(dic.items(), key=lambda item: item[1], reverse=True))
dic = {k: dic[k] for k in list(dic)[:768]} # In order to code the dictionnary on only 2 other bytes with most frequent words
offset = 256
i = 0
for key in dic:
  dic[key] = convertToTenBits(convertToBinary(offset + i))
  offset += 1

print(dic)

reverted_dic = {v: k for k, v in dic.items()}

print(reverted_dic)

{'est': '0100000000', 'ai': '0100000001', 'autre': '0100000010', 'fai': '0100000011', 'peu': '0100000100', 'nouveau': '0100000101', 'grand': '0100000110', 'bon': '0100000111', 'petit': '0100001000', 'm?me': '0100001001', 'va': '0100001010', 'doi': '0100001011', 'di': '0100001100', 'dernier': '0100001101', 'voi': '0100001110', 'seul': '0100001111', 'pren': '0100010000', 'beau': '0100010001', 'sai': '0100010010', 'met': '0100010011', 'veu': '0100010100', 'jeune': '0100010101', 'meilleur': '0100010110', 'français': '0100010111', 'différent': '0100011000', 'trouv': '0100011001', 'nombreux': '0100011010', 'important': '0100011011', 'juste': '0100011100', 'présent': '0100011101', 'donn': '0100011110', 'ven': '0100011111', 'pass': '0100100000', 'fall': '0100100001', 'permett': '0100100010', 'vrai': '0100100011', 'part': '0100100100', 'politique': '0100100101', 'propre': '0100100110', 'social': '0100100111', 'possible': '0100101000', 'général': '0100101001', 'national': '0100101010', 'utilis':

In [20]:
# class to build a tree defining all the possible  string in dic
class SuffixTree():
  class Node():
    def __init__(self,char):
      self.char = char
      self.out = {}
      self.isFinal = False
    def __str__(self, level=0):
      ret = "  " * level + str(self.char) + "\n"
      for c in self.out:
        ret += self.out[c].__str__(level + 1)
      return ret

  def __init__(self, dic):
    self.root = self.Node(None)
    self.root.isFinal = True
    for key in dic:
      cursor = self.root
      for c in key:
        if c in cursor.out:
          cursor = cursor.out[c]
        else:
          node = self.Node(c)
          cursor.out[c] = node
          cursor = node
      cursor.isFinal = True
  def longestPrefix(self, s):
    cursor = self.root
    word = ''
    temp = ''
    end = False
    i = 0
    length = len(s)
    while not(end) and i < length:
      if s[i] in cursor.out:
        temp = temp + s[i]
        cursor = cursor.out[s[i]]
        if cursor.isFinal:
          word = temp
        i += 1
      else:
        end = True
    return word
  def __str__(self):
    return f'{self.root}'

s = SuffixTree({'ab' : 1, 'abc': 2, 'coucou' : 5, 'aba' : 4, 'aaa' : 8})
print(s)
s.longestPrefix('abdc')
    


None
  a
    b
      c
      a
    a
      a
  c
    o
      u
        c
          o
            u



'ab'

In [21]:
tree = SuffixTree(dic)
print(tree)

None
  e
    s
      t
        i
          m
      s
        e
          n
            t
              i
                e
                  l
      p
        é
          r
        r
          i
            t
        a
          c
            e
    u
      r
        o
          p
            é
              e
                n
    n
      f
        a
          n
            t
      t
        e
          n
            d
        i
          e
            r
        r
          e
            p
              r
                i
                  s
                    e
      v
        o
          y
      g
        a
          g
    f
      f
        e
          t
          c
            t
              u
        i
          c
            a
              c
                e
    x
      c
        e
          l
            l
              e
                n
                  t
          p
            t
              i
                o
                  n
                    n
               

In [22]:
# First implementation

string = 'ceci est un test d\'implémentation'

def compress(string, tree, dic):
  i = 0
  ret = ''
  while i<len(string):
    temp = tree.longestPrefix(string[i:])
    n = len(temp)
    if n == 0:
      bit = convertToTenBits(convertToBinary(ord(string[i])))
      ret += bit
      i += 1
    else:
      bit = dic[temp]
      ret += bit
      i += n
  return ret

compressed = compress(string, tree, dic)
print(compressed)

def decompress(string, dic):
  ret = ''
  for i in range(0, len(string) - 1, 10):
    if int(string[i : i + 10], 2) > 255:
      ret += dic[string[i : i + 10]]
    else:
      ret += chr(int(string[i : i + 10], 2))
  return ret

decompressed = decompress(compressed, reverted_dic)
print(decompressed)
  


000110001100011001010001100011000110100100001000000100000000000010000000011101010001101110000010000000011101000100000000000010000000011001000000100111000110100100011011010001110000000110110000111010011001100111000111010000011000010001110100000110100100011011110001101110
ceci est un test d'implémentation
